In [2]:
import numpy as np
import pandas as pd

def generate_temp_data(minutes=500):
    np.random.seed(0)
    base_temp = 25
    temp = [base_temp]

    for _ in range(minutes):
        noise = np.random.normal(0, 0.5)
        change = np.random.choice( [0.1, 0.2, -0.1, -0.2, 0.5, -0.5, 1, -1],p=[0.15, 0.15, 0.15, 0.15, 0.1, 0.1, 0.1, 0.1])
        next_temp = temp[-1] + change + noise
        temp.append(round(next_temp, 2))

    df = pd.DataFrame({'temperature': temp})
    df['minute'] = df.index
    df['prev_temp'] = df['temperature'].shift(1).fillna(method='bfill')
    df['temp_diff'] = df['temperature'] - df['prev_temp']
    df['overheat_alert'] = (df['temperature'] > 30).astype(int)
    return df

df = generate_temp_data()
print(df.head())


   temperature  minute  prev_temp  temp_diff  overheat_alert
0        25.00       0      25.00       0.00               0
1        26.38       1      25.00       1.38               0
2        26.38       2      26.38       0.00               0
3        27.21       3      26.38       0.83               0
4        27.72       4      27.21       0.51               0


C:\Users\arthe\AppData\Local\Temp\ipykernel_15760\997224342.py:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['prev_temp'] = df['temperature'].shift(1).fillna(method='bfill')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   temperature     501 non-null    float64
 1   minute          501 non-null    int64  
 2   prev_temp       501 non-null    float64
 3   temp_diff       501 non-null    float64
 4   overheat_alert  501 non-null    int64  
dtypes: float64(3), int64(2)
memory usage: 19.7 KB


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

df['future_temp'] = df['temperature'].shift(-5).fillna(method='ffill')
df['future_alert'] = (df['future_temp'] > 30).astype(int)

X = df[['temperature', 'prev_temp', 'temp_diff']]
y = df['future_alert']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)

print("Accuracy:", model.score(X_test, y_test))

Accuracy: 0.9900990099009901


C:\Users\arthe\AppData\Local\Temp\ipykernel_15760\2684710677.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['future_temp'] = df['temperature'].shift(-5).fillna(method='ffill')


In [11]:
import time
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# Simulated Real-Time Input with Spikes
def simulate_next_temp(current_temp, step):
    # Every 4th step (i.e., ~2 mins), simulate a possible spike
    if step % 4 == 0 and step != 0:
        return current_temp + np.random.uniform(4, 6)  # spike: +4 to +6 °C
    else:
        change = np.random.choice(
            [0.1, 0.2, -0.1, -0.2, 0.5, -0.5, 1, -1],
            p=[0.15, 0.15, 0.15, 0.15, 0.1, 0.1, 0.1, 0.1]
        )
        noise = np.random.normal(0, 0.4)
        return current_temp + change + noise

# Initial temperature
current_temp = 26.0

print("🔁 Starting real-time prediction with potential spikes...\n")

for step in range(14):  # 14 readings = 7 minutes
    # Generate new reading
    next_temp = round(simulate_next_temp(current_temp, step), 2)
    prev_temp = current_temp
    temp_diff = next_temp - prev_temp
    current_temp = next_temp

    # Model input
    input_data = [[next_temp, prev_temp, temp_diff]]

    # Predict (Assumes model already trained)
    prediction = model.predict(input_data)[0]  # 'model' should be pre-trained

    # Display
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[{timestamp}] Temp: {next_temp:.2f}°C | Δ: {temp_diff:.2f} → ", end='')
    if prediction == 1:
        print("⚠️ ALERT: Overheating expected soon!")
    else:
        print("✅ Status: Normal.")

    time.sleep(2)  # For quick testing. Change to 30 for real-time pacing


🔁 Starting real-time prediction with potential spikes...

[07:59:47] Temp: 27.30°C | Δ: 1.30 → ✅ Status: Normal.
[07:59:49] Temp: 27.79°C | Δ: 0.49 → ✅ Status: Normal.
[07:59:51] Temp: 27.84°C | Δ: 0.05 → ✅ Status: Normal.
[07:59:53] Temp: 27.18°C | Δ: -0.66 → ✅ Status: Normal.
[07:59:55] Temp: 32.95°C | Δ: 5.77 → ⚠️ ALERT: Overheating expected soon!
[07:59:57] Temp: 31.78°C | Δ: -1.17 → ✅ Status: Normal.
[07:59:59] Temp: 31.92°C | Δ: 0.14 → ⚠️ ALERT: Overheating expected soon!
[08:00:01] Temp: 31.91°C | Δ: -0.01 → ⚠️ ALERT: Overheating expected soon!
[08:00:03] Temp: 37.83°C | Δ: 5.92 → ⚠️ ALERT: Overheating expected soon!
[08:00:05] Temp: 38.01°C | Δ: 0.18 → ⚠️ ALERT: Overheating expected soon!
[08:00:07] Temp: 38.01°C | Δ: 0.00 → ⚠️ ALERT: Overheating expected soon!
[08:00:09] Temp: 38.03°C | Δ: 0.02 → ⚠️ ALERT: Overheating expected soon!
[08:00:11] Temp: 43.99°C | Δ: 5.96 → ⚠️ ALERT: Overheating expected soon!
[08:00:13] Temp: 43.49°C | Δ: -0.50 → ⚠️ ALERT: Overheating expected soo